# Google Cloud Platform - Using Machine Learning APIs  ).

This is an upgraded Python revision of [this notebook](https://github.com/GoogleCloudPlatform/training-data-analyst/blob/master/CPB100/lab4c/mlapis.ipynb).

This notebook originally was being processed using DataLab on the Google Cloud Platform.  This particular incarnation of the notebook is for running on Google Colaboratory which I am trying out for the first time.

### Security

First things first - we need to authenticate against the Google Cloud APIs.

#### Getting a Google API Credential.

First, visit <a href="http://console.cloud.google.com/apis">API console</a>, choose "Credentials" on the left-hand menu.  Choose "Create Credentials" and generate an API key for your application. You should probably restrict it by IP address to prevent abuse, but for now, just  leave that field blank and delete the API key after trying out this demo.

Then, when you have your key, you will enter it in this first executable cell:

In [1]:
!pip install -q cloudstorage
!pip install -q webapp2
!pip install --upgrade numpy
!pip install six==1.10.0
!pip install -q datalab
! pip install --upgrade gcs-client


    100% |████████████████████████████████| 13.9MB 2.5MB/s 
  Found existing installation: numpy 1.14.6
    Uninstalling numpy-1.14.6:
      Successfully uninstalled numpy-1.14.6
google-colab 0.0.1a1 has requirement six~=1.11.0, but you'll have six 1.10.0 which is incompatible.
  Found existing installation: six 1.11.0
    Uninstalling six-1.11.0:
      Successfully uninstalled six-1.11.0
    100% |████████████████████████████████| 61kB 4.6MB/s 
    100% |████████████████████████████████| 2.1MB 8.6MB/s 
    100% |████████████████████████████████| 61kB 23.5MB/s 
    100% |████████████████████████████████| 102kB 18.7MB/s 
    100% |████████████████████████████████| 430kB 15.4MB/s 
    100% |████████████████████████████████| 163kB 15.0MB/s 
  Running setup.py bdist_wheel for oauth2client ... - \ done
  Stored in directory: /root/.cache/pip/wheels/7a/0b/de/3f8e939ae5f46f030eee5a16bd2dafde9e1e49ea1480b2f983
  Running setup.py bdist_wheel for pycparser ... - \ done
  Stored in direct

In [2]:
import getpass

APIKEY = getpass.getpass()

··········


In [0]:
import sys
import json
import argparse
import googleapiclient.discovery
import pandas as pd
import skimage.data
from skimage import img_as_float
import numpy as np
import pandas as pd
import os
import sys
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
%matplotlib inline

In [0]:
# Authenticate to GCS.
from google.colab import auth
auth.authenticate_user()

# Create the service client.
from googleapiclient.discovery import build
gcs_service = build('storage', 'v1')


BUCKET = 'wpiopenimageskaggle' 

In [0]:
def create_service():
    return googleapiclient.discovery.build('storage', 'v1')


def list_bucket(bucket):
    """Returns a list of metadata of the objects within the given bucket."""
    service = create_service()

    # Create a request to objects.list to retrieve a list of objects.
    fields_to_return = 'nextPageToken,items(name,size,contentType,metadata(my-key))'
    req = service.objects().list(bucket=bucket, fields=fields_to_return, prefix='Boat')  # returns everything. UrbanSound is top dir in bucket

    all_objects = []
    # If you have too many items to list in one request, list_next() will
    # automatically handle paging with the pageToken.
    while req:
        resp = req.execute()
        all_objects.extend(resp.get('items', []))
        req = service.objects().list_next(req, resp)
    return all_objects

raw_json=json.dumps(list_bucket(BUCKET), indent=2)

In [7]:
test=pd.read_json(raw_json)
print(test[1:2])

  contentType                       name    size
1  image/jpeg  Boat/002200be72145198.jpg  421363


In [14]:
files=test['name']
test_file=test['name'].apply(str)[1]
print(test_file)

Boat/002200be72145198.jpg


In [15]:
from apiclient.http import MediaIoBaseDownload

with open('test_f', 'wb') as f:
  request = gcs_service.objects().get(bucket=BUCKET,
                                            object=test_file)
 
  print(request)
  
  #gray=skimage.data.load(val)
  #print(val)
  media = MediaIoBaseDownload(f, request)

  

  done = False
  while not done:
    # _ is a placeholder for a progress object that we ignore.
    # (Our file is small, so we skip reporting progress.)
    _, done = media.next_chunk()
    
print(type(media))
    

print('Download complete')

<class 'googleapiclient.http.MediaIoBaseDownload'>
Download complete


**BELOW THIS POINT IS CODE TRIED THAT DOESN'T WORK KEPT FOR REFERENCE**

In [0]:
def url_to_image(url):
	# download the image, convert it to a NumPy array, and then read
	# it into OpenCV format
	resp = urllib.urlopen(url)
	image = np.asarray(bytearray(resp.read()), dtype="uint8")
	image = cv2.imdecode(image, cv2.IMREAD_COLOR)
 
	# return the image
	return image

IndentationError: ignored

In [0]:
def url_to_image(url):
	# download the image, convert it to a NumPy array, and then read
	# it into OpenCV format
  with urllib.request.urlopen(url) as file:
      resp=file.read()
      image = np.asarray(bytearray(resp.read()), dtype="uint8")
      image = cv2.imdecode(image, cv2.IMREAD_COLOR)
  return image

In [0]:
with urllib.request.urlopen("http://www.python.org") as url:
    s = url.read()
#I'm guessing this would

In [0]:
bucket = BUCKET
file = test_file
gcs_url = 'https://%(bucket)s.storage.googleapis.com/%(file)s' % {'bucket':bucket, 'file':file}

print(gcs_url)

https://wpiopenimageskaggle.storage.googleapis.com/Boat/002200be72145198.jpg
adc.json  sample_data  test_f


'/content'

In [0]:
# Imports
from google.datalab import Context
from IPython.display import Image

# Define the bucket and and an example image to read
bucket_path = "gs://BUCKET_NAME"
bucket_object = bucket_path + "/google.png"

# List all the objects in your bucket, and read the example image file
%gcs list --objects $bucket_path
%gcs read --object $bucket_object -v img

# Print the image content (see it is in PNG format) and show it
print(type(img))
img
Image(img)

UsageError: Line magic function `%gcs` not found.


In [0]:
# Imports
from google.datalab import Context
from IPython.display import Image

# Define the bucket and and an example image to read
bucket_path = "gs://"+'wpiopenimageskaggle/'
bucket_object = bucket_path + test_file

# List all the objects in your bucket, and read the example image file
%gcs list --objects $bucket_path
%gcs read --object $bucket_object -v img

# Print the image content (see it is in PNG format) and show it
print(type(img))
img
Image(img)

UsageError: Line magic function `%gcs` not found.


In [11]:
from apiclient.http import MediaIoBaseDownload

with open('test_f', 'wb') as f:
  request = gcs_service.objects().get(bucket=BUCKET,
                                            object=test_file)
 
  print(request)
  
  #gray=skimage.data.load(val)
  #print(val)
  media = MediaIoBaseDownload(f, request)

  

  done = False
  while not done:
    # _ is a placeholder for a progress object that we ignore.
    # (Our file is small, so we skip reporting progress.)
    _, done = media.next_chunk()
    
print(type(media))
    

print('Download complete')

NameError: ignored

In [0]:
def import_image(filename):
    #color=skimage.data.load(filename)
    gray=skimage.data.load(filename)
    print(np.shape(gray))
    gray_m=np.asmatrix(gray)
    return(gray_m)

In [0]:
def create_matrix():
    cwd = os.getcwd()
    path=cwd+"\\images"
    files = os.listdir(path)
    gray_data=[]
    filenames=[]
    for infile in files:
        filenames.append(infile[:-4])
        gray=import_image(path+"\\"+infile)
        gray_data.append(gray)
    gray_data=np.array(gray_data)
    print(np.shape(gray_data))
    #print(gray_data)
    return(filenames, gray_data)

create_matrix()

In [0]:
 
def get_image(bucket,file):
    image_vec=[]
    return image_vec


# usage